In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 00. 라이브러리 호출

In [2]:
import os,glob
from PIL import Image
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [3]:
import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.layers import LeakyReLU
import numpy as np

Using TensorFlow backend.


### 01. 경로 지정 및 기타

In [4]:
root_dir = '/content/gdrive/My Drive/image_project/landmark/'

categories = [
"Eiffel Tower"
,"Cristo Redentor"
,"Triumphal Arch"
,"Wànlĭ Chángchéng"
,"Taj Mahal"
,"Sungnyemun Gate"
,"Moai"
,"Seokguram buddha"
,"Golden Gate"
,"Statue of Liberty"
,"Torre di Pisa"
,"Colosseum"
,"Santiago Bernabéu"
,"Sphinx"
,"Burj Khalifa"
,"London Eye"
,"London Tower Bridge" 
] 

nb_classes = len(categories)

### 02. 데이터 로딩 함수 생성

In [5]:
def load_dataset():
    x_train, x_test, y_train, y_test = np.load(root_dir+'landmark_augmentation.npy', allow_pickle=True)
                                               
    print(x_train.shape, y_train.shape)
    
    x_train = x_train.astype('float') / 256
    x_test = x_test.astype('float') / 256
    
    y_train = np_utils.to_categorical(y_train, nb_classes)   # one-hot-encoding
    y_test = np_utils.to_categorical(y_test, nb_classes)
    
    return x_train, x_test, y_train, y_test

### 03-1. 모델 구성 함수 생성
* 검증방법 : 정확도

In [6]:
def build_model(in_shape):
    model = Sequential()
    
    model.add(Convolution2D(32,3,3, border_mode='Same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64,3,3, border_mode='Same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,3,3, border_mode='Same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

### 03-2. 모델 구성 함수 생성
* 검증방법 : 상위 3개 중 정답이 있으면 맞춘 것으로

In [ ]:
from keras import metrics 
import tensorflow as tf
    
def build_model(in_shape):
    model = Sequential()
    
    model.add(Convolution2D(32,3,3, border_mode='Same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64,3,3, border_mode='Same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(128,3,3, border_mode='Same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=3, 
                                                                    name='top_k_categorical_accuracy', 
                                                                    dtype=None)])
    
    return model

### 04. 모델 학습 함수 생성

In [7]:
#from keras.callbacks import EarlyStopping
#epochs=300
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

def model_train(x, y):
    print(x.shape[1:])
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=128, epochs=100)
    
    return model

### 05. 모델 평가 함수 생성


In [8]:
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss = ', score[0])
    print('accuracy = ', score[1])

### 06. 모델 생성, 학습, 평가

In [10]:
# 데이터 불러오기
x_train, x_test, y_train, y_test = load_dataset()

print(x_train.shape, y_train.shape, x_test.shape,  y_test.shape)


(49130, 128, 128, 3) (49130,)
(49130, 128, 128, 3) (49130, 17) (16377, 128, 128, 3) (16377, 17)


In [ ]:
%%time

# 모델 학습
model = model_train(x_train, y_train)

In [ ]:
# 학습이 완료된 모델을 저장

model.save('/content/gdrive/My Drive/image_project/landmark/landmark_model23.h5')

In [11]:
# 테스트 데이터 로딩
def load_test_dataset():
    x_train, x_test, y_train, y_test = np.load('/content/gdrive/My Drive/image_project/landmark/landmark_test.npy', allow_pickle=True)
                                               
    x_test = x_train / 256
    xxx = x_test / 256
    
    y_test = np_utils.to_categorical(y_train, nb_classes)   # one-hot-encoding
    yyy = np_utils.to_categorical(y_test, nb_classes)
    
    return x_test, xxx, y_test, yyy

x_test, xxx, y_test, yyy = load_test_dataset()
print(x_test.shape, y_test.shape)

(152, 128, 128, 3) (152, 17)


In [12]:
# 저장된 모델 불러오기

from tensorflow.python.keras.models import load_model
model = load_model('/content/gdrive/My Drive/image_project/landmark/landmark_model23.h5')

In [13]:
# 모델 평가
model_eval(model, x_test, y_test)

5/5 [==============================] - 0s 35ms/step - loss: 2.1555 - accuracy: 0.8553
loss =  2.1555373668670654
accuracy =  0.8552631735801697
